# Showcasing the SAP Cloud ALM Test Management API

Welcome! This notebook provides a hands-on demonstration of the SAP Cloud ALM Test Management API. We will walk through a complete, story-driven example of how to programmatically create, modify, and manage test cases.

Our story will be:
1.  **Setup & Authentication:** Configure our connection and authenticate with the API.
2.  **Scoping our Work:** Select an SAP Cloud ALM Project and a specific Scope to work within.
3.  **Create a TestCase:** Build a new manual test case from scratch, complete with an initial activity, action, and reference link.
4.  **Enhance the TestCase:** Add more details, such as a new activity and another reference.
5.  **Finalize & Clean Up:** Mark the test case as 'Prepared' and then demonstrate the correct procedures for deletion.

Let's get started!

## 0. Install Dependencies

Before you begin, please run the following code cell to ensure all necessary Python libraries are installed in your environment. This will install `requests` for making API calls, `pandas` for data handling, and `ipywidgets` for the interactive selection elements.

**Note:** After running this cell, you may need to **restart the kernel** and **refresh your browser** for the interactive widgets to display correctly.

In [ ]:
!python -m pip install -q requests pandas ipywidgets requests-oauthlib
print("✅ Dependencies installed and widgets enabled. If widgets do not appear below, please restart the kernel and refresh your browser.")

## 1. Setup and Authentication


---

### Authentication information

You must create a python module file called `./apidata.py` and put the information specific to your tenant there. This includes:

* OAuth2 client ID and secret
* Token url
* Base URL for API calls

Get client ID and secret variables from an external module: this information is sensitive.

These items can be retrieved from the BTP Cockpit 

#### Format of module apidata.py for import

```python
ptm_all_client_id = r'get your client ID from BTP Cockpit'
ptm_all_client_secret = r'get your client secret from BTP Cockpit'
token_url = 'your token url'
base_url = 'your base url'
```


In [ ]:
import apidata as ad

client_id = ad.ptm_all_client_id
client_secret = ad.ptm_all_client_secret
token_url = ad.token_url
base_url = ad.base_url

tm_api_path= "/api/calm-testmanagement/v1/"

print(f"API Base URL: {base_url}")

# --- Header for API Requests ---
headers = {
    "Accept": "application/json",
    "Content-Type": "application/json"
}

### Get Authentication Token
Next, we use our credentials to fetch an OAuth token. This token will be added to the header of all subsequent requests to authenticate our session.

In [ ]:
from requests_oauthlib import OAuth2Session
from oauthlib.oauth2 import BackendApplicationClient

client = BackendApplicationClient(client_id=client_id)
oauth = OAuth2Session(client=client)
token = oauth.fetch_token(token_url=token_url, client_id=client_id,
                          client_secret=client_secret)

# Prepare the authentication header for all subsequent requests
header = {'Authorization': 'Bearer ' + token['access_token']}
print("✅ Authentication successful. Token has been retrieved.")

## 2. Scoping our Work
Before we can create a test case, we need to define where it belongs. This involves selecting a Project and a Scope.

### 2a. Select a Project
Run the cell below to fetch a list of available SAP Cloud ALM projects. Please select one from the list to proceed.

In [ ]:
import requests
import uuid
import pandas as pd

import ipywidgets as widgets
from IPython.display import display, clear_output

# Get All Projects
response = requests.get(f'{base_url}/api/calm-projects/v1/projects', headers=header)

if response.status_code == 200:
    # List the project to select in Grid
    df = pd.json_normalize(response.json())

    # --- Global variable to store selected index ---
    project_id = None

    # --- Header Row ---
    header_ui = widgets.HBox([widgets.HTML(f"<b>{col}</b>") for col in df.columns])

    # --- Radio Buttons ---
    radio = widgets.RadioButtons(
        options=[(f"{row['name']}", i) for i, row in df.iterrows()],
        description='Project:',
        value=None,
        layout={'width': 'auto'}
    )

    # --- Output box ---
    out = widgets.Output()

    def on_change(change):
        global project_id
        if change['new'] is not None:
            selected_index = change['new']
            project_id = df.iloc[selected_index]['id']
            with out:
                out.clear_output(wait=True)
                print(f"✅ Project selected: {df.iloc[selected_index]['name']}")
                print(f"🆔 Project ID: {project_id}")

    radio.observe(on_change, names='value')

    display(widgets.VBox([radio, out]))
else:
    print(f"❌ Failed to fetch projects: {response.status_code} - {response.text}")

### 2b. Select or Create a Scope

Now that a project is selected, we need a scope. A scope defines a specific area of work within the project. You can either select an existing scope from the list or create a new one for this demo.

In [ ]:
# ---------- Global Output Widgets ----------
scope_action_out = widgets.Output()
scope_result_out = widgets.Output()
_last_scope_action = None
scope_id = None  # define globally so it's accessible after selection

# ---------- Function: Select Existing Scope ----------
def select_existing_scope(project_id):
    with scope_action_out:
        clear_output(wait=True)
        print("Fetching available scopes...")

    try:
        scopes_resp = requests.get(
            f"{base_url}/api/calm-processmanagement/v1/scopes?projectId={project_id}",
            headers=header,
            timeout=30
        )

        if scopes_resp.status_code != 200:
            with scope_action_out:
                clear_output(wait=True)
                print(f"❌ Failed to fetch scopes: {scopes_resp.status_code} - {scopes_resp.text}")
            return

        scopes_data = scopes_resp.json().get("value", [])
        if not scopes_data:
            with scope_action_out:
                clear_output(wait=True)
                print("❌ No scopes found for this project. Please create a new one.")
            return

        df_scopes = pd.DataFrame(scopes_data)
        scope_radio = widgets.RadioButtons(
            options=[(f"{row['name']} ({row['id']})", row['id']) for _, row in df_scopes.iterrows()],
            description='Available Scopes:',
            layout={'width': 'auto'}
        )
        confirm_button = widgets.Button(description="Confirm Selection", button_style='success')

        def on_confirm_clicked(b):
            global scope_id
            with scope_result_out:
                clear_output(wait=True)
                scope_id = scope_radio.value
                print(f"✅ Scope Selected: {scope_radio.label}")
                print(f"🆔 Scope ID: {scope_id}")

        confirm_button.on_click(on_confirm_clicked)

        with scope_action_out:
            clear_output(wait=True)
            display(widgets.VBox([scope_radio, confirm_button]))

    except Exception as e:
        with scope_action_out:
            print(f"⚠️ Error fetching scopes: {e}")


# ---------- Function: Create New Scope ----------
def create_new_scope(project_id):
    with scope_action_out:
        clear_output(wait=True)
        print(f"🆕 Creating a new scope for project: {project_id}")
    try:
        scope_body = {"name": f"Notebook Demo Scope - {uuid.uuid4()}", "projectId": project_id}
        created_scope_response = requests.post(
            f"{base_url}/api/calm-processmanagement/v1/scopes",
            json=scope_body,
            headers=header
        )

        if created_scope_response.status_code == 201:
            new_scope = created_scope_response.json()
            global scope_id
            scope_id = new_scope.get("id")
            with scope_result_out:
                clear_output(wait=True)
                print(f"✅ Successfully created new scope: {new_scope.get('name')}")
                print(f"🆔 Scope ID: {scope_id}")
        else:
            with scope_result_out:
                clear_output(wait=True)
                print(f"❌ Failed to create scope: {created_scope_response.status_code} - {created_scope_response.text}")
    except Exception as e:
        with scope_result_out:
            clear_output(wait=True)
            print(f"⚠️ Error creating scope: {e}")

# ---------- Main Logic ----------
def on_scope_action(change):
    action = change.new
    if not project_id:
        with scope_action_out:
            clear_output(wait=True)
            print("⚠️ Please select a project in the step above before choosing an action.")
        return

    if action == "Select Existing Scope":
        select_existing_scope(project_id)
    elif action == "Create New Scope":
        create_new_scope(project_id)

# ---------- Display the UI ----------
scope_action_radio = widgets.RadioButtons(
    options=["Select Existing Scope", "Create New Scope"],
    description="Action:",
    value=None,
    layout={'width': 'auto'}
)
scope_action_radio.observe(on_scope_action, names="value")
ui = widgets.VBox([
    widgets.HTML("<h3>Scope Management</h3>"),
    scope_action_radio,
    scope_action_out,
    scope_result_out
])
display(ui)

## 3. Create a Manual TestCase

With our Project and Scope identified, we can now create our first manual test case. We'll use a `POST` request to the `/ManualTestCases` endpoint. This powerful feature allows us to create the test case and its nested components (activities, actions, Applications, and references) in a single API call.

In [ ]:
if not project_id or not scope_id:
    print("❌ Error: Please ensure you have selected a Project and a Scope in the steps above.")
else:
    body = {
      "title": "Notebook Demo: Initial TestCase",
      "projectId": project_id,
      "scopeId": scope_id,
      "toReferences": [
        {
          "name": "Initial Reference Link",
          "url": "https://help.sap.com/docs/SAP_CLOUD_ALM"
        }
      ],
      "toActivities": [
        {
          "sequence": 1,
           "toApplications": [
            {
              "title": "Fiori App: Test Preparation",
              "url": "https://fiori.apps.library.url"
            }
          ],
          "title": "Activity 1: Setup Test Data",
          "toActions": [
            {
              "sequence": 1,
              "expectedResult": "User is logged in and on the main dashboard.",
              "description": "Log in to the S/4HANA system.",
              "title": "Action 1.1: Log in"
            }
          ]
        }
      ]
    }

    created_mtc_response = requests.post(base_url + tm_api_path + "ManualTestCases", headers=header, json=body, timeout=30)
    
    if created_mtc_response.status_code == 201:
        resp = created_mtc_response.json()
        manual_test_case_id = resp.get('uuid')
        
        print(f"✅ TestCase created successfully! Status: {created_mtc_response.status_code}")
        print(f"🆔 Manual TestCase ID: {manual_test_case_id}")
        
        # Store the ID of the first activity for later use
        first_activity_id = resp.get('toActivities', [{}])[0].get('uuid')
        print(f"🆔 First Activity ID: {first_activity_id}")
    else:
        print(f"❌ Failed to create TestCase. Status: {created_mtc_response.status_code}")
        print(f"Response: {created_mtc_response.text}")

## 3b. Create a Manual TestCase with Process Relations

To create a test case that is linked to specific SAP Cloud ALM business processes, we need to gather four essential identifiers that establish the process hierarchy:

1. **`solutionProcessId`** - The ID of the specific business process
2. **`solutionProcessFlowId`** - The ID of the process flow within that process
3. **`solutionProcessFlowDiagramId`** - The ID of the specific diagram within the process flow
4. **`contentPackageId`** - The content package identifier (typically "CUSTOM" for custom processes)

These process relations allow test cases to be directly linked to business processes in SAP Cloud ALM, enabling better traceability and process coverage analysis. In the following cells, we'll demonstrate how to discover these IDs dynamically and create a process-linked test case.

### Step 1: Discover Available Custom Solution Processes

First, we'll fetch all available solution processes for our selected project and scope, then filter for those that are actually in scope (`isScoped: true`). This ensures we only work with processes that are relevant to our current project scope.

In [ ]:
import json

if not project_id or not scope_id:
    print("❌ Error: Please ensure you have selected a Project and a Scope in the steps above.")
else:
    # Get all Custom Solution Processes for the project and scope
    print("🔍 Fetching custom solution processes...")
    response = requests.get(
        f"{base_url}/api/calm-processmanagement/v1/solutionProcesses",
        params={
            "projectId": project_id,
            "scopeId": scope_id,
            "solutionScenarioId" : "CUSTOM",
            "$top": 500
        },
        headers=header,
        timeout=30
    )
    
    if response.status_code != 200:
        print(f"❌ Failed to fetch solution processes. Status: {response.status_code}")
        print(f"Response: {response.text}")
    else:
        response_data = response.json()
        all_processes = response_data.get("value", [])
        
        # Filter for items with isScoped: true (as requested by user)
        scoped_processes = [item for item in all_processes if item.get("isScoped") == True]
        
        print(f"📊 Total processes found: {len(all_processes)}")
        print(f"✅ Scoped processes (isScoped: true): {len(scoped_processes)}")
        
        if scoped_processes:
            print("\n🎯 Available scoped processes:")
            for i, process in enumerate(scoped_processes[:5]):  # Show first 5
                print(f"  {i+1}. {process.get('solutionProcessId')} - {process.get('solutionProcessVersionName', 'N/A')}")
            
            # Select the first scoped process for demonstration
            selected_process = scoped_processes[0]
            solutionProcessId = selected_process.get("solutionProcessId")
            contentPackageId = selected_process.get("solutionScenarioId", "CUSTOM")
            
            print(f"\n✅ Selected process: {solutionProcessId}")
            print(f"📦 Content package: {contentPackageId}")
        else:
            print("⚠️ No scoped processes found. Cannot proceed with process-linked test case creation.")
            solutionProcessId = None

### Step 2: Get Solution Process Flow

With a solution process selected, we now need to retrieve its associated process flows. Each solution process can have multiple flows that represent different execution paths or variants.

In [ ]:
if not solutionProcessId:
    print("❌ No solution process available. Skipping process flow retrieval.")
    solutionProcessFlowId = None
else:
    print(f"🔍 Fetching process flows for solution process: {solutionProcessId}")
    
    flow_url = f"{base_url}/api/calm-processauthoring/v1/solutionProcesses/{solutionProcessId}/solutionProcessFlows"
    
    response = requests.get(flow_url, headers=header, timeout=30)
    
    if response.status_code != 200:
        print(f"❌ Failed to fetch process flows. Status: {response.status_code}")
        print(f"Response: {response.text}")
        solutionProcessFlowId = None
    else:
        response_data = response.json()
        flows = response_data.get("value", [])
        
        print(f"📊 Found {len(flows)} process flow(s)")
        
        if flows:
            # Select the first process flow
            selected_flow = flows[0]
            solutionProcessFlowId = selected_flow.get("id")
            
            print(f"✅ Selected (first) process flow ID: {solutionProcessFlowId}")
        else:
            print("❌ No process flows found for this solution process.")
            solutionProcessFlowId = None

### Step 3: Get Solution Process Flow Diagram

Finally, we need to retrieve the specific diagram within the process flow. The diagram represents the visual representation of the process steps and is the most granular level for linking test cases.

In [ ]:
if not solutionProcessFlowId:
    print("❌ No process flow available. Skipping diagram retrieval.")
    solutionProcessFlowDiagramId = None
else:
    print(f"🔍 Fetching process flow diagrams for flow: {solutionProcessFlowId}")
    
    diagram_url = f"{base_url}/api/calm-processauthoring/v1/solutionProcessFlows/{solutionProcessFlowId}/solutionProcessFlowDiagrams"
    
    response = requests.get(diagram_url, headers=header, timeout=30)
    
    if response.status_code != 200:
        print(f"❌ Failed to fetch process flow diagrams. Status: {response.status_code}")
        print(f"Response: {response.text}")
        solutionProcessFlowDiagramId = None
    else:
        response_data = response.json()
        diagrams = response_data.get("value", [])
        
        print(f"📊 Found {len(diagrams)} diagram(s)")
        
        if diagrams:
            # Select the first diagram
            selected_diagram = diagrams[0]
            solutionProcessFlowDiagramId = selected_diagram.get("id")
            
            print(f"✅ Selected (first) diagram ID: {solutionProcessFlowDiagramId}")
        else:
            print("❌ No diagrams found for this process flow.")
            solutionProcessFlowDiagramId = None

### Step 4: Create Process-Linked Manual Test Case

Now that we have collected all the required process identifiers, we can create a manual test case that is directly linked to the SAP Cloud ALM business process. This linkage provides full traceability between test cases and business processes.

In [ ]:
if not project_id or not scope_id:
    print("❌ Error: Please ensure you have selected a Project and a Scope in the steps above.")
elif not all([solutionProcessId, solutionProcessFlowId, solutionProcessFlowDiagramId]):
    print("❌ Error: Missing required process identifiers. Cannot create process-linked test case.")
    print(f"   - Solution Process ID: {solutionProcessId}")
    print(f"   - Process Flow ID: {solutionProcessFlowId}")
    print(f"   - Flow Diagram ID: {solutionProcessFlowDiagramId}")
else:
    print("🚀 Creating process-linked manual test case...")
    print(f"🔗 Process Links:")
    print(f"   - Solution Process: {solutionProcessId}")
    print(f"   - Process Flow: {solutionProcessFlowId}")
    print(f"   - Flow Diagram: {solutionProcessFlowDiagramId}")
    print(f"   - Content Package: {contentPackageId}")
    
    body = {
        "title": "Notebook Demo: Process-Linked TestCase",
        "projectId": project_id,
        "scopeId": scope_id,
        "solutionProcessId": solutionProcessId,
        "solutionProcessFlowId": solutionProcessFlowId,
        "solutionProcessFlowDiagramId": solutionProcessFlowDiagramId,
        "contentPackageId": contentPackageId,
        "toReferences": [
            {
                "name": "SAP Cloud ALM Documentation",
                "url": "https://help.sap.com/docs/SAP_CLOUD_ALM"
            }
        ],
        "toActivities": [
            {
                "sequence": 1,
                "title": "Activity 1: Process Setup",
                "toApplications": [
                    {
                        "title": "SAP Fiori Launchpad",
                        "url": "https://fiori.apps.library.url"
                    }
                ],
                "toActions": [
                    {
                        "sequence": 1,
                        "title": "Action 1.1: Login to System",
                        "description": "Log in to the SAP system using your credentials.",
                        "expectedResult": "User is successfully logged in and sees the main dashboard."
                    }
                ]
            }
        ]
    }

    try:
        created_mtc_response = requests.post(
            base_url + tm_api_path + "ManualTestCases", 
            headers=header, 
            json=body, 
            timeout=30
        )
        
        if created_mtc_response.status_code == 201:
            resp = created_mtc_response.json()
            manual_test_case_id = resp.get('uuid')
            
            print(f"✅ Process-linked TestCase created successfully!")
            print(f"🆔 Manual TestCase ID: {manual_test_case_id}")
            print(f"📋 Title: {resp.get('title')}")
            
            # Store the ID of the first activity for later use
            first_activity_id = resp.get('toActivities', [{}])[0].get('uuid')
            print(f"🆔 First Activity ID: {first_activity_id}")
            
        else:
            print(f"❌ Failed to create process-linked TestCase. Status: {created_mtc_response.status_code}")
            print(f"Response: {created_mtc_response.text}")
            
    except requests.exceptions.RequestException as e:
        print(f"❌ Network error while creating test case: {e}")

## 4. Enhance the TestCase
Our test case has been created, but it's common to need to add or modify details later. Here, we'll demonstrate how to add a new activity and a new reference.

### 4a. Add a New Activity with Steps
Let's add a second activity to our test case. We do this by posting to the `toActivities` navigation property of our specific `ManualTestCase`.

In [ ]:
if not manual_test_case_id:
    print("❌ Error: manual_test_case_id not found. Please create a TestCase first.")
else:
    activity_body = {
      "title": "Activity 2: Execute Test Scenario",
      "sequence": 2,
      "parent_ID": manual_test_case_id,
      "toActions": [
        {
          "sequence": 1,
          "expectedResult": "The sales order is created successfully.",
          "description": "Navigate to 'Create Sales Order' app and fill in required data.",
          "title": "Action 2.1: Create Sales Order"
        },
        {
          "sequence": 2,
          "expectedResult": "The sales order status is 'Completed'.",
          "description": "Verify the created sales order in the 'Manage Sales Orders' app.",
          "title": "Action 2.2: Verify Sales Order"
        }
      ]
    }

    add_activity_response = requests.post(base_url + tm_api_path + f"ManualTestCases/{manual_test_case_id}/toActivities", headers=header, json=activity_body, timeout=30)
    
    if add_activity_response.status_code == 201:
        print(f"✅ New activity added successfully! Status: {add_activity_response.status_code}")
        new_activity_id = add_activity_response.json().get('uuid')
        print(f"🆔 New Activity ID: {new_activity_id}")
    else:
        print(f"❌ Failed to add new activity. Status: {add_activity_response.status_code}")
        print(f"Response: {add_activity_response.text}")

### 4b. Add a New Reference
Similarly, we can add more reference links to our test case by posting to the `toReferences` navigation property.

In [ ]:
if not manual_test_case_id:
    print("❌ Error: manual_test_case_id not found. Please create a TestCase first.")
else:
    reference_body = {
        "name": "Project Documentation",
        "url": "https://documentation.example.com/project-x",
        "parentTestCase_ID" : manual_test_case_id
    } 

    add_reference_response = requests.post(base_url + tm_api_path + f"ManualTestCases/{manual_test_case_id}/toReferences", headers=header, json=reference_body, timeout=30)
    
    if add_reference_response.status_code == 201:
        print(f"✅ New reference added successfully! Status: {add_reference_response.status_code}")
        new_reference_id = add_reference_response.json().get('uuid')
        print(f"🆔 New Reference ID: {new_reference_id}")
    else:
        print(f"❌ Failed to add new reference. Status: {add_reference_response.status_code}")
        print(f"Response: {add_reference_response.text}")

### 4c. Set Activities 'In Scope'
By default, activities might be created as 'out of scope'. To include them in your formal test plan and execution, you need to set their `isInScope` property to `True`. We'll do this now for the two activities we created using a `PATCH` request for each.

In [ ]:
activity_ids_to_update = []
if 'first_activity_id' in locals() and first_activity_id:
    activity_ids_to_update.append(first_activity_id)
if 'new_activity_id' in locals() and new_activity_id:
    activity_ids_to_update.append(new_activity_id)

if not activity_ids_to_update:
    print("⚠️ No activity IDs found. Please ensure the previous creation steps were successful.")
else:
    print("Updating isInScope flag for created activities...")
    for activity_id in activity_ids_to_update:
        # First, get the latest version of the activity to retrieve its ETag
        get_activity_response = requests.get(base_url + tm_api_path + f"Activities/{activity_id}", headers=header, timeout=30)
        
        if get_activity_response.status_code == 200:
            etag = get_activity_response.json().get('modifiedAt')
            patch_header = header.copy()
            patch_header["If-Match"] = etag
            patch_body = {"isInScope": True}
            
            # Send the PATCH request
            update_response = requests.patch(
                base_url + tm_api_path + f"Activities/{activity_id}",
                headers=patch_header,
                json=patch_body,
                timeout=30
            )
            
            if update_response.status_code == 200:
                print(f"✅ Successfully set isInScope=True for Activity ID: {activity_id}")
            else:
                print(f"❌ Failed to update Activity ID {activity_id}. Status: {update_response.status_code}, Response: {update_response.text}")
        else:
            print(f"❌ Could not retrieve Activity ID {activity_id} to get ETag. Status: {get_activity_response.status_code}")

## 5. Finalize and Clean Up

The final steps in our story are to mark the test case as ready for execution and then to demonstrate the proper deletion procedures.

### 5a. Set TestCase to 'Prepared'
A test case must be in a 'Prepared' state before it can be executed. We'll use a `PATCH` request to update its title and set the `isPrepared` flag to `True`. Note that `PATCH` requests require an `If-Match` header containing the entity's last modified timestamp to prevent concurrent modification conflicts.

In [ ]:
if not manual_test_case_id:
    print("❌ Error: manual_test_case_id not found. Please create a TestCase first.")
else:
    # First, get the latest version of the test case to retrieve its ETag (modifiedAt)
    get_mtc_response = requests.get(base_url + tm_api_path + "ManualTestCases/" + manual_test_case_id, headers=header, timeout=30)
    
    if get_mtc_response.status_code == 200:
        etag = get_mtc_response.json().get('modifiedAt')
        patch_header = header.copy()
        patch_header["If-Match"] = etag
        
        # Prepare the body for the PATCH request
        patch_body = {
            "title": "Notebook Demo: Final TestCase (Prepared)",
            "isPrepared": True
        }
        
        # Send the PATCH request
        patched_mtc_response = requests.patch(base_url + tm_api_path + "ManualTestCases/" + manual_test_case_id, headers=patch_header, json=patch_body, timeout=30)
        
        if patched_mtc_response.status_code == 200:
            print(f"✅ TestCase updated and set to 'Prepared'. Status: {patched_mtc_response.status_code}")
            print(f"New Title: {patched_mtc_response.json()['title']}")
            print(f"Is Prepared: {patched_mtc_response.json()['isPrepared']}")
        else:
            print(f"❌ Failed to update TestCase. Status: {patched_mtc_response.status_code}")
            print(f"Response: {patched_mtc_response.text}")
    else:
        print(f"❌ Failed to retrieve TestCase for ETag. Status: {get_mtc_response.status_code}")

### 5b. Deletion Rules: Standard vs. Force Delete

The API provides two ways to delete a test case, which depend on its execution history and require different authorization scopes.

#### Standard DELETE Operation
- **Required Scope:** `calm-api.testcases.delete`
- **Use Case:** Delete test cases that have **never been executed**
- **Behavior:** Removes the test case and all its associated data (activities, actions, references, applications)

- **Safety:** Will return a `412 Precondition Failed` error if the test case has associated test runs or results, preventing accidental data lossFor this demo, let's assume our test case has been executed in the UI. We will first attempt a standard delete to see the expected error, and then use the force delete action to clean up properly.



#### Force Delete Operation

- **Best Practice:** Apply the principle of least privilege - only grant this scope when standard delete operations are insufficient
- **Required Scope:** `calm-api.testcases.force-delete`
- **HIGH RISK**- **Limited Use Cases:** Only use when you specifically need to delete test cases with execution history (e.g., data migration scenarios, administrative cleanup of test environments)

- **Use Case:** Delete test cases that **have been executed** and contain test runs or results
- **Compliance Impact:** Deleted execution data cannot be recovered and may be required for audit trails

- **Behavior:** Permanently removes the test case and **all associated execution data** including test runs, test results, execution history, and all nested entities
- **Data Loss Risk:** This operation is **irreversible** and will permanently delete test execution history



**CRITICAL SECURITY WARNING:** The `calm-api.testcases.force-delete` scope should **only be granted if absolutely necessary** and with careful consideration:

#### Attempting Standard `DELETE` (Expected to Fail)

This call will fail because we are simulating that the test case has already been executed.

In [ ]:
if not manual_test_case_id:
    print("❌ Error: manual_test_case_id not found. Please run the creation steps first.")
else:
    # Get the latest ETag for the If-Match header
    get_mtc_response = requests.get(base_url + tm_api_path + "ManualTestCases/" + manual_test_case_id, headers=header, timeout=30)
    
    if get_mtc_response.status_code == 200:
        etag = get_mtc_response.json().get('modifiedAt')
        delete_header = header.copy()
        delete_header["If-Match"] = etag

        # Attempt the standard delete
        mtc_delete_response = requests.delete(base_url + tm_api_path + f"ManualTestCases/{manual_test_case_id}", headers=delete_header, timeout=30)
        
        print(f"Standard Delete Status: {mtc_delete_response.status_code}")
        if mtc_delete_response.status_code == 412:
            print("✅ As expected, the standard delete failed because the TestCase has execution data.")
            print(f"Response: {mtc_delete_response.text}")
        else:
            print(f"⚠️ Unexpected response. Status: {mtc_delete_response.status_code}, Response: {mtc_delete_response.text}")
    else:
         print(f"❌ Failed to retrieve TestCase for deletion. Status: {get_mtc_response.status_code}")

#### Using `forceDelete` Action

Now we use the `forceDeletionIncludingTestRunsAndResults` action to correctly delete the test case and its history.

**Important:** This operation requires the `calm-api.testcases.force-delete` scope (which we included in our initial authorization setup). If this scope is missing, you will receive a `403 Forbidden` error.

In [ ]:
if not manual_test_case_id:
    print("❌ Error: manual_test_case_id not found. Please run the creation steps first.")
else:
    # Get the latest ETag for the If-Match header
    get_mtc_response = requests.get(base_url + tm_api_path + "ManualTestCases/" + manual_test_case_id, headers=header, timeout=30)
    
    # Check if the test case still exists before trying to delete
    if get_mtc_response.status_code == 200:
        etag = get_mtc_response.json().get('modifiedAt')
        force_delete_header = header.copy()
        force_delete_header["If-Match"] = etag
        
        # Trigger the force delete action
        force_delete_response = requests.post(base_url + tm_api_path + f"ManualTestCases/{manual_test_case_id}/api.v1.ExternalServiceAPI.forceDeletionIncludingTestRunsAndResults", headers=force_delete_header, timeout=30)
        
        print(f"Force Delete Action Status: {force_delete_response.status_code}")
        if force_delete_response.status_code == 200:
            print("✅ TestCase and all its execution data have been successfully deleted.")
            print(f"Response: {force_delete_response.text}")
        elif force_delete_response.status_code == 403:
            print("❌ Force delete failed: Insufficient authorization.")
            print("💡 Make sure your API service includes the 'calm-api.testcases.force-delete' scope.")
            print(f"Response: {force_delete_response.text}")
        else:
             print(f"❌ Force delete failed. Status: {force_delete_response.status_code}, Response: {force_delete_response.text}")
    elif get_mtc_response.status_code == 404:
        print("ℹ️ TestCase was already deleted in a previous step.")
    else:
        print(f"❌ Failed to retrieve TestCase for deletion. Status: {get_mtc_response.status_code}")

# Appendix: Additional API Capabilities

This section contains examples of other API requests that were not used in the main storyline. They demonstrate further capabilities for managing individual entities like Activities, Actions, and References.

### A1. ManualTestCase Operations

#### Retrieve Lists of Nested Entities
You can retrieve lists of entities associated with a test case, such as its references, activities, or tags.

In [ ]:
# Note: These calls will fail if the TestCase was deleted. They are for demonstration purposes.
import json

print("--- Retrieving References for a TestCase ---")
read_references_response = requests.get(base_url + tm_api_path + f"ManualTestCases/{manual_test_case_id}/toReferences", headers=header, timeout=30)
print(f"Status: {read_references_response.status_code}")
if read_references_response.ok: print(f"Response: {json.dumps(read_references_response.json(), indent=2)}")

print("--- Retrieving Activities for a TestCase ---")
read_activities_response = requests.get(base_url + tm_api_path + f"ManualTestCases/{manual_test_case_id}/toActivities", headers=header, timeout=30)
print(f"Status: {read_activities_response.status_code}")
if read_activities_response.ok: print(f"Response: {json.dumps(read_activities_response.json(), indent=2)}")

print("--- Retrieving Tag Assignments for a TestCase ---")
read_tags_response = requests.get(base_url + tm_api_path + f"ManualTestCases/{manual_test_case_id}/toTagAssignments", headers=header, timeout=30)
print(f"Status: {read_tags_response.status_code}")
if read_tags_response.ok: print(f"Response: {json.dumps(read_tags_response.json(), indent=2)}")

#### Deep Updates are Not Allowed
The API for updating a `ManualTestCase` (`PATCH /ManualTestCases/{uuid}`) only allows for updates to the direct properties of the test case itself (like `title` or `isPrepared`). You cannot update nested entities (like `toActivities` or `toReferences`) through this endpoint. This is by design to ensure data consistency. You must use the dedicated endpoints for those entities as shown in the main story (e.g., `POST /ManualTestCases/{uuid}/toActivities`).

In [ ]:
print("Demonstrating that deep updates via PATCH on ManualTestCase are ignored.")
# This body attempts to update the title AND a nested reference, but only the title will be changed.
mtc_patch_body =  {
  "title": "Attempting a deep update",
  "toReferences": [
    {
      "name": "This-Reference-Update-Will-Be-Ignored",
      "url": "https://example.com/ignored"
    }
  ]
}

get_mtc_response = requests.get(base_url + tm_api_path + "ManualTestCases/" + manual_test_case_id, headers=header)
if get_mtc_response.status_code == 200:
    patch_header = header.copy()
    patch_header["If-Match"] = get_mtc_response.json().get('modifiedAt')
    patch_response = requests.patch(base_url + tm_api_path + "ManualTestCases/" + manual_test_case_id, headers=patch_header, json=mtc_patch_body)
    print(f"Patch Request Status: {patch_response.status_code}")
    import json
    if patch_response.ok: print(f"Response Body: {json.dumps(patch_response.json(), indent=2)}")
else:
    print(f"Could not fetch TestCase to run demo. Status: {get_mtc_response.status_code}")

### A2. Activity Operations
Manage individual Activities outside the context of a deep create.

#### Retrieve a Single Activity by ID
Demonstrate how to fetch a specific Activity using its UUID. We'll use the first activity created in the main storyline.

In [ ]:
# Note: The 'activity_id' used here is from the main storyline. This may fail if that section was not run.
activity_id_for_appendix = first_activity_id if 'first_activity_id' in locals() else 'demo-activity-id'
import json

print(f"--- Retrieving a single Activity by ID: {activity_id_for_appendix} ---")
get_activity_response = requests.get(base_url + tm_api_path + f"Activities/{activity_id_for_appendix}", headers=header, timeout=30)
print(f"Status: {get_activity_response.status_code}")
if get_activity_response.ok: 
    print(f"Response: {json.dumps(get_activity_response.json(), indent=2)}")

#### Update an Activity
Demonstrate how to update an Activity's properties using a PATCH request with the required If-Match header.

In [ ]:
print(f"--- Updating an Activity by ID: {activity_id_for_appendix} ---")
if get_activity_response.ok:
    patch_header = header.copy()
    patch_header["If-Match"] = get_activity_response.json().get('modifiedAt')
    activity_update_body = {"title": "Updated Activity Title via Appendix"}
    update_activity_response = requests.patch(base_url + tm_api_path + f"Activities/{activity_id_for_appendix}", headers=patch_header, json=activity_update_body, timeout=30)
    print(f"Status: {update_activity_response.status_code}")
    if update_activity_response.ok: 
        print(f"Response: {json.dumps(update_activity_response.json(), indent=2)}")
else:
    print("⚠️ Cannot update activity - failed to retrieve it in the previous step.")

#### Delete an Activity
Demonstrate how to delete an Activity. Note that we need to fetch the latest ETag before deletion.

In [ ]:
print(f"--- Deleting an Activity by ID: {activity_id_for_appendix} ---")
# To delete, we need the latest ETag again
get_activity_response_pre_delete = requests.get(base_url + tm_api_path + f"Activities/{activity_id_for_appendix}", headers=header, timeout=30)
if get_activity_response_pre_delete.ok:
    delete_header = header.copy()
    delete_header["If-Match"] = get_activity_response_pre_delete.json().get('modifiedAt')
    delete_activity_response = requests.delete(base_url + tm_api_path + f"Activities/{activity_id_for_appendix}", headers=delete_header, timeout=30)
    print(f"Status: {delete_activity_response.status_code}")  # Expect 204 No Content on success
else:
    print(f"⚠️ Cannot delete activity - failed to retrieve it for ETag. Status: {get_activity_response_pre_delete.status_code}")

### A3. Action & Application Operations
Manage individual Actions and Applications. Note that an Application can only be added to an Activity if one does not already exist.

#### Create Temporary Activity for Demonstrations
To demonstrate individual Action and Application operations, we need an existing activity. Let's create a temporary one.

In [ ]:
import json

print("--- Creating a temporary Activity to host an Action ---")
temp_activity_body = {"title": "Temp Activity for Appendix Demo", "sequence": 99, "parent_ID": manual_test_case_id}
temp_activity_response = requests.post(base_url + tm_api_path + f"ManualTestCases/{manual_test_case_id}/toActivities", headers=header, json=temp_activity_body)
temp_activity_id = None
if temp_activity_response.ok:
    temp_activity_id = temp_activity_response.json().get('uuid')
    print(f"Created temporary activity with ID: {temp_activity_id}")
else:
    print(f"Failed to create temporary activity. Status: {temp_activity_response.status_code}")

#### Create and Delete an Action
Demonstrate creating an Action for the temporary Activity and then deleting it.

In [ ]:
if temp_activity_id:
    print("--- Creating a new Action for the temporary Activity ---")
    action_body = {"title": "Appendix Action", "description": "...", "expectedResult" : "...", "sequence": 1, "parent_ID" : temp_activity_id}
    create_action_response = requests.post(base_url + tm_api_path + f"Activities/{temp_activity_id}/toActions", headers=header, json=action_body)
    print(f"Status: {create_action_response.status_code}")
    if create_action_response.ok:
        print(f"Response: {json.dumps(create_action_response.json(), indent=2)}")
        action_id = create_action_response.json().get('uuid')
        print(f"Created Action with ID: {action_id}")
        
        print(f"--- Deleting the Action with ID: {action_id} ---")
        delete_action_header = header.copy()
        delete_action_header["If-Match"] = create_action_response.json().get('modifiedAt')
        delete_action_response = requests.delete(base_url + tm_api_path + f"Actions/{action_id}", headers=delete_action_header)
        print(f"Status: {delete_action_response.status_code}")
    else:
        print(f"Failed to create action. Response: {create_action_response.text}")
else:
    print("⚠️ No temporary activity ID available. Please run the previous cell first.")

#### Attempt to Create an application
Demonstrate attempting to create an application. Note that only one application can be associated with an Activity, so this may fail if one already exists.

In [ ]:
if temp_activity_id:
    print("--- Attempting to create an application for the temporary Activity ---")
    application_body = {"title": "Appendix application", "url": "https://example.com/app", "parent_ID": temp_activity_id}
    create_app_response = requests.post(base_url + tm_api_path + f"Activities/{temp_activity_id}/toApplications", headers=header, json=application_body)
    print(f"Status: {create_app_response.status_code}")
    # Note: If an application was created with the test case, a second one cannot be added, resulting in a 400 error.
    if create_app_response.ok:
        print(f"Response: {json.dumps(create_app_response.json(), indent=2)}")
    else: 
        print(f"Response: {create_app_response.text}")
else:
    print("⚠️ No temporary activity ID available. Please run the previous cells first.")

### A4. General List Endpoints
You can also query the top-level endpoints for entities like `/Activities`, `/Actions`, etc., across all test cases you have access to. These endpoints support powerful OData query options like `$filter`, `$top`, `$skip`, and `$orderby`.

#### Retrieve All Activities
Query the top-level `/Activities` endpoint to get a list of all activities across test cases you have access to.

In [ ]:
import json

print("--- Retrieving a list of all Activities (first page) ---")
all_activities_response = requests.get(base_url + tm_api_path + "Activities?$top=5", headers=header, timeout=30)
print(f"Status: {all_activities_response.status_code}")
if all_activities_response.ok: 
    print(f"Response: {json.dumps(all_activities_response.json(), indent=2)}")

#### Retrieve All Actions
Query the top-level `/Actions` endpoint to get a list of all actions across test cases you have access to.

In [ ]:
print("--- Retrieving a list of all Actions (first page) ---")
all_actions_response = requests.get(base_url + tm_api_path + "Actions?$top=5", headers=header, timeout=30)
print(f"Status: {all_actions_response.status_code}")
if all_actions_response.ok: 
    print(f"Response: {json.dumps(all_actions_response.json(), indent=2)}")

#### Retrieve All Tag Assignments
Query the top-level `/TagAssignments` endpoint to get a list of all tag assignments across test cases you have access to.

In [ ]:
print("--- Retrieving a list of all Tag Assignments (first page) ---")
all_tags_response = requests.get(base_url + tm_api_path + "TagAssignments?$top=5", headers=header, timeout=30)
print(f"Status: {all_tags_response.status_code}")
if all_tags_response.ok: 
    print(f"Response: {json.dumps(all_tags_response.json(), indent=2)}")